<a href="https://colab.research.google.com/github/MohamedWael45/RFM/blob/main/RFM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import os
url="https://raw.githubusercontent.com/MohamedWael45/RFM/main/RFMwithExcel.xlsx"
df=pd.read_excel(url)


In [8]:
df_second_page = pd.read_excel(url, sheet_name="Customer RFM Analysis")
df_second_page.head()

,NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9


In [12]:
df_second_page = df_second_page.dropna()
df_second_page.head()

,NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9


In [15]:
column_names = df_second_page.columns.tolist()
print("Column Names:")
for column_name in column_names:
    print(column_name)

Column Names:
NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.
Unnamed: 1
Unnamed: 2
Unnamed: 3
Unnamed: 4
Unnamed: 5
Unnamed: 6
Unnamed: 7


In [16]:
new_column_names = [
    "CustomerID", "MostRecentOrderDate", "DaysSinceLastOrder",
    "SalesOrderCount", "TotalSalesAmount", "Recency", "Frequency", "Monetary"]
df_second_page.columns = new_column_names
df_second_page.head()

,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9


In [18]:
df_second_page = df_second_page.drop(df_second_page.index[0])
df_second_page.head()

,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9
6,11004,2013-05-01 00:00:00,290,3,8196.01,2,9,9


In [73]:
df_second_page['DaysSinceLastOrder'] = pd.to_numeric(df_second_page['DaysSinceLastOrder'], errors='coerce')


df_second_page['TotalSalesAmount'] = df_second_page['TotalSalesAmount'].round().astype(int)


try:
  df_second_page['SalesOrderCount'] = pd.to_numeric(df_second_page['SalesOrderCount'], errors='coerce')
except ValueError:
  print("Warning: 'SalesOrderCount' column contains non-numeric values and cannot be converted to int.")


print(df_second_page.dtypes)



CustomerID              object
MostRecentOrderDate     object
DaysSinceLastOrder       int64
SalesOrderCount          int64
TotalSalesAmount         int64
Recency                 object
Frequency               object
Monetary                object
Recency2               float64
Frequency2               int64
Monetary2              float64
dtype: object


In [77]:
def assign_score(value):
    if value >= 0.8:
        return 5
    elif value >= 0.6:
        return 4
    elif value >= 0.4:
        return 3
    elif value >= 0.2:
        return 2
    else:
        return 1

# Apply pd.qcut to bin values into percentile ranges and assign scores
for col in ['DaysSinceLastOrder', 'SalesOrderCount', 'TotalSalesAmount']:
    percentiles = pd.qcut(df_second_page[col], 5, labels=False, duplicates='drop')  # Compute percentiles
    scores = percentiles.apply(lambda x: assign_score(x / 5))  # Map percentiles to scores
    df_second_page[col + '_score'] = scores  # Assign scores to new column

# Rename the new columns
df_second_page.rename(columns={'DaysSinceLastOrder_score': 'R2', 'SalesOrderCount_score': 'F2', 'TotalSalesAmount_score': 'M2'}, inplace=True)

print(df_second_page)

      CustomerID  MostRecentOrderDate  DaysSinceLastOrder  SalesOrderCount  \
2          11000  2013-05-03 00:00:00                 288                3   
3          11001  2013-12-10 00:00:00                  67                3   
4          11002  2013-02-23 00:00:00                 357                3   
5          11003  2013-05-10 00:00:00                 281                3   
6          11004  2013-05-01 00:00:00                 290                3   
...          ...                  ...                 ...              ...   
18481      29479  2012-09-04 00:00:00                 529                1   
18482      29480  2013-07-17 00:00:00                 213                1   
18483      29481  2011-08-13 00:00:00                 917                1   
18484      29482  2012-09-18 00:00:00                 515                1   
18485      29483  2012-09-09 00:00:00                 524                1   

       TotalSalesAmount Recency Frequency Monetary  Recency2  F